# A Statistical Analysis of Medicare Claims Data

Medicare is single-payer health insurance program provided by the government mainly for Americans aged 65 and older. It is an important social insurance program with more than 50 million people. 

In this notebook, we will look at the publicly available claims data from the Center of Medicare services. The data can be downloaded from their [website](https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/DE_Syn_PUF.html) where claims between 2008 and 2010 are split into 20 samples. Since the whole data is quite large, we will only focus on one of the samples (Sample 1).

This is a continuation of a previous [exploratory analysis](https://github.com/bhimmetoglu/springboard-exercises/blob/master/data_story_telling/Medicare%20Data%20Exploration.ipynb). 



## Data loading and cleaning

This part is idential to the previous notebook. 

In [1]:
# imports
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
sns.set_style('whitegrid')

# Load Beneficiary data
beneficiaries = pd.read_csv("./data/DE1_0_2008_Beneficiary_Summary_File_Sample_1.zip")

## Rename some columns
rename_dict = {"BENE_BIRTH_DT":"birthday", "BENE_DEATH_DT":"deathday", "BENE_SEX_IDENT_CD":"sex",
               "BENE_RACE_CD":"race", "BENE_ESRD_IND":"renal_disease", "SP_STATE_CODE":"state",
               "BENE_COUNTY_CD":"county", "BENE_HI_CVRAGE_TOT_MONS":"partA", 
               "BENE_SMI_CVRAGE_TOT_MONS":"partB", "BENE_HMO_CVRAGE_TOT_MONS":"hmo",
               "PLAN_CVRG_MOS_NUM":"partD", "SP_ALZHDMTA":"alzheimer", "SP_CHF":"hearth_failure",
               "SP_CHRNKIDN":"kidney_disease", "SP_CNCR":"cancer", "SP_COPD":"pulmanory_disease",
               "SP_DEPRESSN":"depression","SP_DIABETES":"diabetes","SP_ISCHMCHT":"ismechic_hearth",
               "SP_OSTEOPRS":"osteoporosis", "SP_RA_OA":"rheumatoid","SP_STRKETIA":"stroke"}

## List of conditions
diseases = ['renal_disease', 'alzheimer', 'hearth_failure', 'kidney_disease', 'cancer', 'pulmanory_disease',
            'depression', 'diabetes', 'ismechic_hearth', 'osteoporosis', 'rheumatoid','stroke']

beneficiaries = beneficiaries.rename(columns=rename_dict)

# One-hot encoding
beneficiaries['renal_disease'] = beneficiaries['renal_disease'].map(lambda x: 1 if x=="Y" else 0)

for disease in diseases[1:]:
    beneficiaries[disease] = beneficiaries[disease].map(lambda x: 1 if x==1 else 0)
    
## Process Dates
# Birth dates
beneficiaries['birthday'] = pd.to_datetime(beneficiaries['birthday'].astype(str))

# Death dates: first get dead beneficiaries and then set their death time
dead_bene = beneficiaries[~pd.isnull(beneficiaries['deathday'])]['deathday'].astype(int).astype(str)
beneficiaries.loc[dead_bene.index,'deathday'] = pd.to_datetime(dead_bene)

## Process Payments
# Inpatient
beneficiaries['tot_inpatient'] = beneficiaries['MEDREIMB_IP'] + beneficiaries['BENRES_IP'] + beneficiaries['PPPYMT_IP']

# Outpatient
beneficiaries['tot_outpatient'] = beneficiaries['MEDREIMB_OP'] + beneficiaries['BENRES_OP'] + beneficiaries['PPPYMT_OP']

# Carrier
beneficiaries['tot_carrier'] = beneficiaries['MEDREIMB_CAR'] + beneficiaries['BENRES_CAR'] + beneficiaries['PPPYMT_CAR']

## Demographic features
# Convert to one-hot: M=1/F=0
beneficiaries['sex'] = beneficiaries['sex'].map(lambda x: 1 if x==1 else 0)

# Convert race encoding into clear text
def race_code(x):
    if (x == 1):
        return "white"
    elif (x==2):
        return "black"
    elif (x == 5):
        return "hispanic"
    elif (x == 3):
        return "other"

beneficiaries['race'] = beneficiaries['race'].map(lambda x: race_code(x))

## Statistical Analysis


### Analysis of costs by condition
Let's first look at top two diseases with highest costs of treatment: `stroke` and `cancer`. The null hypothesis is that there is no difference between the average costs: $ H_0:\,\,{\bar X} = {\bar Y}$ where X reprsents `stroke` and Y represents `cancer`. The alternative is that the average costs are different. 


While the costs are not normally distributed, their averages are nearly normally distributed by CLT. Therefore, we can use the t-test (or z-test):

In [2]:
df_stroke = beneficiaries[beneficiaries['stroke'] == 1]
stroke_costs = df_stroke['tot_carrier'] + df_stroke['tot_inpatient'] + df_stroke['tot_outpatient']

df_cancer = beneficiaries[beneficiaries['cancer'] == 1]
cancer_costs = df_cancer['tot_carrier'] + df_cancer['tot_inpatient'] + df_cancer['tot_outpatient']

In [3]:
stats.ttest_ind(stroke_costs, cancer_costs, equal_var=False)

Ttest_indResult(statistic=12.555963587226978, pvalue=6.9760597642885558e-36)

The p-value is very small, so we reject the null hypothesis. This is what we have also seen in the exploratory analysis: There is a difference between the mean costs of stroke and cancer.

Let's perform another test using bootstrap resampling and create confidence intervals for the mean and the median difference between cancer and stroke costs:

In [4]:
# Set number of simulations and resmaple from stroke & cancer
nsim = 1000
medians_ = np.zeros(nsim)
means_ = np.zeros(nsim)
for s in range(nsim):
    stroke_rs = np.random.choice(stroke_costs, size=len(stroke_costs), replace=True)
    cancer_rs = np.random.choice(cancer_costs, size=len(cancer_costs), replace=True)
    means_[s] = np.mean(stroke_rs) - np.mean(cancer_costs)
    medians_[s] = np.median(stroke_rs) - np.median(cancer_costs)
    

Using the resamples, we compute the bootsrap estimate of ${\rm med}[X-Y$] and ${\hat se}_{{\rm med}[X-Y]}$. Then, we construct a 95% confidence interval (CI):

In [5]:
# Difference of median costs
medians_avg = np.mean(medians_) # Bootstrap estimate of median differences
medians_std = np.std(medians_)  # Bootstrap estimate of standard err. of median differences
ts = (medians_ - medians_avg)/medians_std 
low = medians_avg - np.percentile(ts,97.5)*medians_std
high = medians_avg + np.percentile(ts,97.5)*medians_std
print("CI: median (stroke-cancer) cost: [{:.2f},{:.2f}]".format(low,high))

CI: median (stroke-cancer) cost: [2931.43,4135.00]


Now we do the same for the boostrap estimate of ${\bar X}-{\bar Y}$ and ${\bar se}_{{\bar X}-{\bar Y}}$

In [6]:
# Difference of mean costs
means_avg = np.mean(means_) # Bootstrap estimate of mean differences
means_std = np.std(means_)  # Bootstrap estimate of standard err. of median differences
ts = (means_ - means_avg)/means_std
low = means_avg - np.percentile(ts,97.5)*means_std
high = means_avg + np.percentile(ts,97.5)*means_std
print("CI: mean (stroke-cancer) cost: [{:.2f},{:.2f}]".format(low,high))

CI: mean (stroke-cancer) cost: [4513.29,5802.20]


These clearly show that the mean and median cost of stroke is higher than that of cancer treamnet. Now, let's look at some effect sizes to check whether this is actually significant.

We can use overlap and Cohen's d as an effect size:

In [7]:
# Compute threshold and overlap
s_stroke = stroke_costs.std()
s_cancer = cancer_costs.std()
mu_stroke = stroke_costs.mean()
mu_cancer = cancer_costs.mean()
n_stroke = len(stroke_costs)
n_cancer = len(cancer_costs)

# Pooled variance
S_p = np.sqrt( ((n_stroke-1)*s_stroke**2 + (n_cancer-1)*s_cancer**2 ) / (n_stroke+n_cancer-2) )

threshold = (s_stroke * mu_cancer + s_cancer * mu_stroke)/(s_cancer + s_stroke)
stroke_below = np.sum(stroke_costs.values < threshold)
cancer_above = np.sum(cancer_costs > threshold)

stroke_overlap = stroke_below / n_stroke
cancer_overlap = cancer_above / n_cancer
misclassification_rate = 0.5*(stroke_overlap + cancer_overlap)
print("Misclassification rate = {:.2f}".format(misclassification_rate))

# Cohen's d
d = (mu_stroke - mu_cancer)/S_p
print("Cohen's d = {:.2f}".format(d))

Misclassification rate = 0.45
Cohen's d = 0.23


These effect sizes are small. This shows that while the statistical tests provide evidence for a larger cost for stroke than cancer, the difference may not be very significant.

### Analysis of conditions by gender

Let's consider how medical conditions are distributed among male and female patients.

In [8]:
df = beneficiaries[diseases + ['sex']]
df2 = pd.melt(df, value_vars=diseases, value_name = 'has_disease', var_name='disease', id_vars = 'sex')
df3 = df2.groupby(['sex','disease'])['has_disease'].sum().unstack(0)
df3

sex,0,1
disease,,
alzheimer,12943,9467
cancer,4123,3292
depression,14568,10272
diabetes,25499,18561
hearth_failure,19046,14109
ismechic_hearth,27966,20976
kidney_disease,10729,7957
osteoporosis,11858,8319
pulmanory_disease,9095,6648


Now, let's check whether there is an actual difference between the distribution of diseases between male (coded 0) and female (coded 1) patients. 

Our null hypothesis is that the probability of all the diseases are equal between male and female. The alternative is that at least two are not equal. We can apply the chi-square test:

In [9]:
chi2, p_val, dof, _ = stats.chi2_contingency(np.array(df3.values))
print("P-value = {:.2e}".format(p_val))

P-value = 8.29e-07


The p-value is very small, thus we reject the null hypothesis. Below, we will look at effect sizes for ismechic hearth disease and diabetes. 

### Analysis of conditions by race

Let's apply the same test we used for male/female difference to distribution of diseases among races:

In [11]:
df = beneficiaries[diseases + ['race']]
df2 = pd.melt(df, value_vars=diseases, value_name = 'has_disease', var_name='disease', id_vars = 'race')
df3 = df2.groupby(['race','disease'])['has_disease'].sum().unstack(0)
df3

race,black,hispanic,other,white
disease,,,,
alzheimer,2203,474,755,18978
cancer,685,123,219,6388
depression,2546,567,849,20878
diabetes,4294,935,1553,37278
hearth_failure,3297,682,1072,28104
ismechic_hearth,4706,988,1635,41613
kidney_disease,1972,355,584,15775
osteoporosis,1900,372,682,17223
pulmanory_disease,1566,332,545,13300


In [12]:
chi2, p_val, dof, _ = stats.chi2_contingency(np.array(df3.values))
print("P-value = {:.2e}".format(p_val))

P-value = 8.78e-10


Again, we reject the null hypothesis and at least two diseases are distributed differently between the races. Below, we will look at effect sizes for ismechic hearth disease and diabetes. 

### Analsysis of ismechic hearth disease and diabetes by gender

Let's now concentrate on ismechic heart disease and diabetes, the top two medical conditions.

First let's look at **ismechic heart disease** :

In [13]:
df = beneficiaries[['ismechic_hearth' , 'sex']]
s1 = df.groupby('sex')['ismechic_hearth'].sum()
s2 = df.groupby('sex').size()
df2 = pd.DataFrame({'disease':s1,'no_disease':s2-s1})
df2

,disease,no_disease
sex,,
0,27966,36381
1,20976,31029


Let's apply the two sample binomial test. The null hypothesis is that there is no difference between the probabilitis of ismechic hearth disease between males and females.

In [14]:
n_m = s2[0]
n_f = s2[1]
hat_p_m = df2.loc[0]['disease']/n_m
hat_p_f = df2.loc[1]['disease']/n_f

se = np.sqrt(hat_p_m*(1-hat_p_m)/n_m + hat_p_f*(1-hat_p_f)/n_f)
TS = ( hat_p_m - hat_p_f )/se
print("Test statistic = {:.2f}".format(TS))

Test statistic = 10.76


In [15]:
p_val = stats.norm.sf(TS)
print("p-value = {:.2e}".format(p_val))

p-value = 2.71e-27


so we can reject the null hypothesis. Male patients seem to have higher rates of ismechic hearth disease than female patients. Let's also compute the estimate for the odds ratio:

$$ OR = \frac{p_m/(1-p_m)}{p_f/(1-p_f)} $$

where $p_{m(f)}$ is the probability that a (fe)male patient has disease.

In [16]:
odds_ratio = (hat_p_m/(1-hat_p_m))/(hat_p_f/(1-hat_p_f))
odds_ratio

1.1371056726305639

meaning that male patients are 1.14 times more likely to have ismechic hearth disease than females. This is not necessarily a very significant difference.

Now let's do the same exercise for **diabetes** :

In [17]:
df = beneficiaries[['diabetes' , 'sex']]
s1 = df.groupby('sex')['diabetes'].sum()
s2 = df.groupby('sex').size()
df2 = pd.DataFrame({'disease':s1,'no_disease':s2-s1})
df2

,disease,no_disease
sex,,
0,25499,38848
1,18561,33444


In [18]:
n_m = s2[0]
n_f = s2[1]
hat_p_m = df2.loc[0]['disease']/n_m
hat_p_f = df2.loc[1]['disease']/n_f

se = np.sqrt(hat_p_m*(1-hat_p_m)/n_m + hat_p_f*(1-hat_p_f)/n_f)
TS = ( hat_p_m - hat_p_f )/se
print("Test statistic = {:.2f}".format(TS))

Test statistic = 13.80


In [19]:
p_val = stats.norm.sf(TS)
print("p-value = {:.2e}".format(p_val))

p-value = 1.19e-43


so we can reject the null hypothesis. Male patients seem to have higher rates of diabetes than female patients. Let's also compute the estimate for the odds ratio:

In [20]:
odds_ratio = (hat_p_m/(1-hat_p_m))/(hat_p_f/(1-hat_p_f))
odds_ratio

1.1826910979310661

meaning that male patients are 1.18 times more likely to have diabetes than females. Again, this is not necessarily a very significant difference.

### Analsysis of ismechic hearth disease and diabetes by race

Let's repeat the same exercise for investigating the distributions of these conditions between races:

First, we look at **ismechic heart disease** :

In [21]:
df = beneficiaries[['ismechic_hearth','race']]
s1 = df.groupby('race')['ismechic_hearth'].sum()
s2 = df.groupby('race').size()
df2 = pd.DataFrame({'disease':s1,'no_disease':s2-s1})
df2

,disease,no_disease
race,,
black,4706,7637
hispanic,988,1741
other,1635,3296
white,41613,54736


Since we have three groups here, we can either perform the tests between pairs or use chi-square test. Let's use the latter approach:

In [22]:
chi2, p_val, dof, _ = stats.chi2_contingency(np.array(df2.values))
print("P-value = {:.2e}".format(p_val))

P-value = 1.06e-70


meaning that at least of the the races have a different distribution of diseases. From the table let's estimate probabilities:

In [23]:
p_black = df2.loc['black']['disease'] / s2['black']
p_hispanic = df2.loc['hispanic']['disease'] / s2['hispanic']
p_other = df2.loc['other']['disease'] / s2['other']
p_white = df2.loc['white']['disease'] / s2['white']
print("p_black = {:.2f}, p_hispanic = {:.2f}, p_other = {:.2f}, p_white = {:.2f}".format(p_black,
                                                                                        p_hispanic,
                                                                                        p_other,
                                                                                        p_white))

p_black = 0.38, p_hispanic = 0.36, p_other = 0.33, p_white = 0.43


Black and white patients seem to have the highest probabilities. Let's compute the odds ratio for them:

In [24]:
odds_ratio = (p_white/(1-p_white))/(p_black/(1-p_black))
odds_ratio

1.2337490673464673

meaning, white patients are 1.23 times more likely to have ismechic hearth disease than black patients. Again, this is not a very large effect.

Let's repeat the same exercise for **diabetes** :

In [25]:
df = beneficiaries[['diabetes','race']]
s1 = df.groupby('race')['diabetes'].sum()
s2 = df.groupby('race').size()
df2 = pd.DataFrame({'disease':s1,'no_disease':s2-s1})
df2

,disease,no_disease
race,,
black,4294,8049
hispanic,935,1794
other,1553,3378
white,37278,59071


In [26]:
chi2, p_val, dof, _ = stats.chi2_contingency(np.array(df2.values))
print("P-value = {:.2e}".format(p_val))

P-value = 2.82e-38


In [27]:
p_black = df2.loc['black']['disease'] / s2['black']
p_hispanic = df2.loc['hispanic']['disease'] / s2['hispanic']
p_other = df2.loc['other']['disease'] / s2['other']
p_white = df2.loc['white']['disease'] / s2['white']
print("p_black = {:.2f}, p_hispanic = {:.2f}, p_other = {:.2f}, p_white = {:.2f}".format(p_black,
                                                                                        p_hispanic,
                                                                                        p_other,
                                                                                        p_white))

p_black = 0.35, p_hispanic = 0.34, p_other = 0.31, p_white = 0.39


In [28]:
odds_ratio = (p_white/(1-p_white))/(p_black/(1-p_black))
odds_ratio

1.1829276093880128

Similar to ismechic hearth disease, the p-value suggest different distribution of diabetes between races. 
However, the odds ratio is not significantly large: White patients are 1.19 times more likely to have diabetes than black patients. Again, this is not a very large effect.

## Conclusions

We have made a simple statistical analysis of the medicare costs data and came up with the following conclusions:

1. The highest costs of treatment are due to stroke and cancer. While stroke has a higher mean cost than cancer, the difference has a small effect size.
2. Overall, diseases are distributed differently between races and sex. This is established by the chi-squared test
3. Male patients have a larger probability of having ismechic hearth disease and diabetes (the two most common conditions) than females. However, the odds ratio is only slighlty larger than 1, so the effect is not very significant. 
4. White patients have a larger probability of having ismechic hearth disease and diabetes than blacks. However, the odds ratio is only slighlty larger than 1, so the effect is not very significant. 